In [12]:
import pandas as pd
import requests
import io
import datetime
from datetime import datetime

climate_url = 'https://github.com/ncaroppiOC/ATG-2006_Wk5_CropCalculator/blob/main/GrowthStages.csv'
growth_url = 'https://github.com/ncaroppiOC/ATG-2006_Wk5_CropCalculator/blob/main/GrowthStages.csv'


Climate_Stations_df = pd.read_csv(climate_url)
Growth_df = pd.read_csv(growth_url)

Climate_Stations_df.index = Climate_Stations_df['STATION_NAME']
Province_List = Climate_Stations_df['ENG_PROV_NAME'].unique().sort()
Crop_list = list(Growth_df['CROP'].unique())


ParserError: Error tokenizing data. C error: Expected 1 fields in line 41, saw 10


In [ ]:
def requestCrop():

	for i, crop in enumerate(Crop_list):
		print(str(i), '\t', crop)

	
	num = input('Select Crop (enter a value between 0-12)')
	num = int(num)

	print(f'You Selected: \t {Crop_list[num]}')
	print('\n')
	
	return num

In [ ]:
def requestProv():

	for i, prov in enumerate(Province_List):
		print(str(i), '\t', prov)
	
	num = input('Select province: (enter a value between 0-12)')
	num = int(num)

	print(f'You Selected: \t {Province_List[num]}')
	print('\n')
	
	return num

In [ ]:
def requestSeedDate():

	year = int(input('Input seeding year (4 digits):'))
	month = int(input('Input seeding month (1-2 digits):'))
	day = int(input('Input seeding day (1-2 digits):'))

	SeedingDate = datetime(year, month, day)

	print(f'Your seeding date is: {SeedingDate}')
	print('\n')

	return SeedingDate
	

In [ ]:
def requestClimateStation(StationList):

	numStations = len(StationList)

	for i, station in enumerate(StationList):
			print(str(i), '\t', station)

	print('\n')
	num = int(input(f'Select climate station: (enter a value between 0-{numStations-1})'))
	print('\n')
	print(f'You Selected: \t {StationList[num]}')

	return num

In [ ]:
SeedingDate = requestSeedDate()

Crop_num = requestCrop()
Crop_type = Crop_list[Crop_num]


Prov_num = requestProv()
Province = Province_List[Prov_num]

Climate_Stations_list= list(Climate_Stations_df[Climate_Stations_df['ENG_PROV_NAME'] == Province]['STATION_NAME'])

stationNum = requestClimateStation(Climate_Stations_list)
selectedStation = Climate_Stations_list[stationNum]

Your seeding date is: 2024-04-08 00:00:00


0 	 Sunflower
1 	 Peas
2 	 Lentils
3 	 Chick Pea (Desi)
4 	 Mustard (S. Alba)
5 	 Mustard (B. juncea)
6 	 Canola(B. rapa)
7 	 Canola (B. napus)
8 	 Flax
9 	 Canary Seed
10 	 Oats
11 	 Wheat (Hard Red)
12 	 Barley
You Selected: 	 Barley


0 	 MANITOBA
1 	 NEWFOUNDLAND
2 	 SASKATCHEWAN
3 	 ALBERTA
4 	 NOVA SCOTIA
5 	 PRINCE EDWARD ISLAND
6 	 ONTARIO
7 	 QUEBEC
8 	 BRITISH COLUMBIA
9 	 YUKON TERRITORY
10 	 NORTHWEST TERRITORIES
11 	 NUNAVUT
12 	 NEW BRUNSWICK
You Selected: 	 SASKATCHEWAN


0 	 KELLIHER
1 	 ROCKGLEN (AUT)
2 	 NIPAWIN
3 	 SOUTHEND
4 	 MELFORT
5 	 SPIRITWOOD WEST
6 	 LEROY
7 	 BEECHY
8 	 CORONACH SPC
9 	 BUTTE ST PIERRE
10 	 ROSETOWN EAST
11 	 MOOSE JAW A
12 	 BUFFALO NARROWS (AUT)
13 	 SCOTT CDA
14 	 WYNYARD (AUT)
15 	 ASSINIBOIA AIRPORT
16 	 VAL MARIE SOUTHEAST
17 	 COLLINS BAY CAMECO
18 	 URANIUM CITY (AUT)
19 	 LUCKY LAKE
20 	 LA RONGE RCS
21 	 KIPLING
22 	 OXBOW
23 	 SWIFT CURRENT CDA
24 	 SONNINGDALE
25 	 WASKESIU LAKE
26 	 W

In [ ]:
def getStationMetaData(StationName):

	CLIMATE_ID = Climate_Stations_df.loc[StationName, 'CLIMATE_IDENTIFIER']
	LAT = Climate_Stations_df.loc[StationName, 'y']
	LONG = Climate_Stations_df.loc[StationName, 'x']
	PROVINCE = Climate_Stations_df.loc[StationName, 'ENG_PROV_NAME']
	
	return CLIMATE_ID, LAT, LONG, PROVINCE

In [ ]:
CLIMATE_ID, LAT, LONG, PROVINCE = getStationMetaData(selectedStation)

In [ ]:
def getClimateData(CLIMATE_IDENTIFER):
    
    API_URL = 'https://api.weather.gc.ca/collections/climate-daily/items?f=csv&lang=en-CA&limit=5000&CLIMATE_IDENTIFIER=' + CLIMATE_IDENTIFER +'&datetime=2013-01-01T00:00:00Z/2023-12-12T12:31:12Z&sortby=LOCAL_DATE'
	
    # IO library StringIO method is required to convert string into a file-like object for pandas to parse it.

    response = requests.get(API_URL)

    if response.status_code != 200:
        print('API request failed with status code {respones.status_code}')
        return None
    
    else: 
        print('Request successful')
        
        response_CSV = io.StringIO(response.text)

        Climate_df = pd.read_csv(response_CSV)

    return Climate_df



In [ ]:
def findAverageGDD(dataFrame):
	dataFrame['LOCAL_DATE'] = pd.to_datetime(dataFrame['LOCAL_DATE'])
	dataFrame = dataFrame.sort_values(['LOCAL_DATE'])
	dataFrame.index = dataFrame['LOCAL_DATE']
	dataFrame = dataFrame.drop(columns=['TOTAL_SNOW_FLAG', 
							'MAX_REL_HUMIDITY', 
							'MAX_TEMPERATURE_FLAG', 
							'TOTAL_PRECIPITATION_FLAG', 
							'COOLING_DEGREE_DAYS', 
							'HEATING_DEGREE_DAYS_FLAG', 
							'MIN_TEMPERATURE_FLAG', 
							'MEAN_TEMPERATURE_FLAG', 
							'ID', 
							'MAX_REL_HUMIDITY_FLAG', 
							'SPEED_MAX_GUST_FLAG', 
							'MIN_REL_HUMIDITY_FLAG', 
							'CLIMATE_IDENTIFIER', 
							'TOTAL_RAIN', 
							'SNOW_ON_GROUND', 
							'DIRECTION_MAX_GUST', 
							'COOLING_DEGREE_DAYS_FLAG', 
							'TOTAL_PRECIPITATION', 
							'TOTAL_RAIN_FLAG', 
							'HEATING_DEGREE_DAYS', 
							'DIRECTION_MAX_GUST_FLAG', 
							'MIN_REL_HUMIDITY', 
							'SNOW_ON_GROUND_FLAG', 
							'TOTAL_SNOW', 
							'SPEED_MAX_GUST', 
							'MEAN_TEMPERATURE', 
							'LOCAL_MONTH', 
							'LOCAL_DAY', 
							])
	
	dataFrame['DAY_OF_YEAR'] = dataFrame.index.day_of_year

	# Calculating GDD
	dataFrame['MEAN_TEMP'] = (dataFrame['MAX_TEMPERATURE'] + dataFrame['MIN_TEMPERATURE'])/2
	dataFrame['MEAN_TEMP'] = dataFrame['MEAN_TEMP'].fillna(method='ffill')
	dataFrame['GDD'] = dataFrame['MEAN_TEMP'].apply(lambda x: x if x >= 0 else 0)
	dataFrame['Cumulative_GDD'] = dataFrame.groupby('LOCAL_YEAR')['GDD'].cumsum()

	stationName = dataFrame['STATION_NAME'][1]
	
	# Pivoting data and calculating averages
	df_yearly = dataFrame.pivot(index= 'DAY_OF_YEAR', columns= 'LOCAL_YEAR', values= 'Cumulative_GDD')
	df_yearly = df_yearly.fillna(method='ffill')
	df_yearly['AVERAGE'] = df_yearly.mean(axis=1)

	AverageGDD = df_yearly['AVERAGE']
	AverageGDD.name = stationName

	return dataFrame, AverageGDD

In [ ]:
Climate_data = getClimateData(CLIMATE_ID)

Request successful


EmptyDataError: No columns to parse from file

In [ ]:
def getDOY(climate_series, GDD_Avg, SeedingDate):

	SeedingDOY = int(SeedingDate.strftime('%j'))
	SeedingGDD = climate_series[SeedingDOY]

	climate_series_adjusted = climate_series - SeedingGDD

	Expected_DOYs = []
	
	for value in GDD_Avg:
		filtered_series = climate_series_adjusted[climate_series_adjusted > value]
		if len(filtered_series) > 0:
			earliest_date = filtered_series.idxmin()
		else:
			earliest_date = 0
		
		Expected_DOYs.append(earliest_date)

	Expected_DOYs = pd.Series(Expected_DOYs)

	return Expected_DOYs

In [ ]:
Climate_data, GDD_Series = findAverageGDD(Climate_data)
YEAR = SeedingDate.year

Growth_df['DOY'] = getDOY(GDD_Series, Growth_df['GDD_AVG'], SeedingDate)
Growth_df['DATE'] = pd.to_datetime(Growth_df['DOY'], format='%j', errors='coerce').dt.strftime(f'{YEAR}-%m-%d')

In [ ]:
Schedule_DF = Growth_df[Growth_df['CROP'].str.startswith(Crop_type)]

if Growth_df['DATE'].isnull().any():
	print(f'There are not sufficient Growing Degree Days to grow {Crop_type} to maturity in {selectedStation}, {PROVINCE} when planted on {SeedingDate}')

else:
	print(f'Estimated Growth Stages for {Crop_type} planted on {SeedingDate} in {selectedStation}, {Province} ')
	for index, row in Schedule_DF.iterrows():
		stage = Schedule_DF.loc[index]['STAGE']
		descript = Schedule_DF.loc[index]['DESCRIPTION']
		date_ymd = Schedule_DF.loc[index]['DATE']
		# y = Schedule_DF.loc[index]['GDD_AVG']
		print(f'Growth Stage: {stage}\tEstimated Date: {date_ymd}\t{descript}')